Séries Históricas: http://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/series-historicas/

Lyaout do Arquivo: http://www.b3.com.br/data/files/C8/F3/08/B4/297BE410F816C9E492D828A8/SeriesHistoricas_Layout.pdf

In [1]:
#https://www.youtube.com/watch?v=Znpbz6RzLFI&feature=youtu.be
#https://albertosouza.net/artigos/22-importando-dados-bovespa
#http://www.bmfbovespa.com.br/pt_br/servicos/market-data/historico/mercado-a-vista/cotacoes-historicas/
    
import pandas as pd
from decimal import Decimal
import os.path
from os import path

# Functions

In [2]:
def Str(line, start, end):
    return line[start-1:end].strip()

def Value(line, start, end):
    string = Str(line, start, end)
    value = string[0:len(string)-2]
    digit = string[len(string)-2:len(string)]
    valueStr = value + '.' + digit
    return Decimal(valueStr)

def DateStr(line, start, end):
    string = Str(line, start, end)
    dateformat = '{}/{}/{}'.format(string[6:8], string[4:6], string[0:4])
    return dateformat
    
def Check(line):
    if (Str(line, 11, 12) == "02"):
        if (Str(line, 25, 27) == "010"):
            return True
    return False

def FormatLineCsv(data):
    n = len(data)
    i = -1
    line = ''
    for value in data:
        i+=1
        if(i>0):
            line+=','
        line+=str(value)
    return line

def find_csv_number_lines(path, ismax=False):
    rows = []
    for file in os.listdir(path):
        with open(path+'/'+file) as f:
            lines = sum(1 for line in f)
            rows.append([file, lines])

    csv = pd.DataFrame(rows, columns=['File', 'Lines'])
    if (ismax==False):
        return csv;
    maxlines = csv['Lines'].max()
    return csv.loc[(csv['Lines'] == maxlines)]

def deletecsv(folder, filesNames):
    for file in filesNames:
        delete_file(folder+'/'+file)

def delete_file(file, verbose=False):
    if os.path.exists(file):
        os.remove(file)
    elif (verbose):
        print("File doesn't exists")

def clear_foler(path):
    for file in os.listdir(path):
        delete_file(path + file)

def extract_historic(from_file, path_to):
    path_to += '/'
    looping=-1
    symbol_list=[]
    header = FormatLineCsv(['Codneg','Data','Abertura','Maxima','Minima','Fechamento','Empresa'])
    clear_foler(path_to)
    
    with open(from_file, "r") as f:            
        for line in f:
            looping+=1

            # header .txt b3, next
            if(looping==0):
                continue

            if(Check(line)):
                rows=[]
                codneg = Str(line, 13, 24)#CODNEG - CÓDIGO DE NEGOCIAÇÃO DO PAPEL                        
                rows.append(codneg)
                rows.append(DateStr(line, 3, 10))#DATA DO PREGÃO
                rows.append(Value(line, 57, 69))#PREABE - PREÇO DE ABERTURA DO PAPELMERCADO NO PREGÃO
                rows.append(Value(line, 70, 82))#PREMAX - PREÇO MÁXIMO DO PAPELMERCADO NO PREGÃO
                rows.append(Value(line, 83, 95))#PREMIN - PREÇO MÍNIMO DO PAPELMERCADO NO PREGÃO
                rows.append(Value(line, 109, 121))#PREULT - PREÇO DO ÚLTIMO NEGÓCIO DO PAPEL-MERCADO NO PREGÃO
                rows.append(Str(line, 28, 39))#NOMRES - NOME RESUMIDO DA EMPRESA EMISSORA DO PAPEL

                # format values
                row_format = FormatLineCsv(rows) + '\n'

                file = path_to + codneg + '.csv'
                # check if not exists file to symbol
                # if not exists, create file with header
                if (codneg not in symbol_list):
                    symbol_list.append(codneg)
                    with open(file, "a") as f:
                        f.write(header+'\n')

                # write info to file
                with open(file, "a") as f:
                    f.write(row_format)
        return len(symbol_list)

# Gera dados

In [3]:
folder_to = "datasets-b3/"
file_from = "datasets/series_historicas/COTAHIST_A2020.TXT"

Extrai os dados e gerar 1 csv para cada ativo

In [4]:
count = extract_historic(file_from, folder_to)
print('{} csv criados com sucesso!!!'.format(count))

972 csv criados com sucesso!!!


Mantém somente os csv que contém mais registros e deleta o resto

In [5]:
listcsv = find_csv_number_lines(folder_to)

In [6]:
number_max = listcsv['Lines'].max()
files = listcsv.loc[(listcsv['Lines'] < number_max)]['File'].tolist()
deletecsv(folder_to, files)
print('{} csv deletados com sucesso!!!'.format(len(files)))

709 csv deletados com sucesso!!!
